In [18]:
import pandas as pd
from unidecode import unidecode

tabela = pd.read_excel('variáveis.xlsx')
tabela = tabela.tail(69)
tabela['Município'] = tabela['Município'].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
tabela['Município'] = tabela['Município'].str[:-5]
tabela['Partido Governador'] = tabela['Partido Governador'].str[5:]
tabela['Partido Governador'] = tabela['Partido Governador'].astype(str)
display(tabela)
print(tabela['Ano de Criação'].unique())

,Ano de Criação,Unidade de Conservação,Categoria de Manejo,Orgão Gestor,Esfera Administrativa,Estado,Município,Aporte Populacional,Partido Governador,Partido Prefeito,Há alinhamento?
199,2006,RESERVA EXTRATIVISTA DO RECANTO DAS ARARAS DE ...,Reserva Extrativista,Instituto Chico Mendes de Conservação da Biodi...,Federal,GO,Sao Domingos,NaN,PP,PSDB\nPT,NaN
200,2006,RESERVA EXTRATIVISTA LAGO DO CEDRO,Reserva Extrativista,Instituto Chico Mendes de Conservação da Biodi...,Federal,GO,Aruana,NaN,PP,PSDC (DC),NaN
201,2006,PARQUE ESTADUAL DA MATA ATLÂNTICA,Parque,Secretaria Estadual do Meio Ambiente e dos Rec...,Estadual,GO,Agua Limpa,NaN,nan,?,NaN
202,2006,ÁREA DE RELEVANTE INTERESSE ECOLÓGICO DA CABEC...,Área de Relevante Interesse Ecológico,Secretaria de Meio Ambiente e Agricultura Sust...,Municipal,GO,Alto Paraiso de Goias,NaN,PP,PFL (DEM),NaN
203,2006,ÁREA DE PROTEÇÃO AMBIENTAL VARGEM DAS FLORES,Área de Proteção Ambiental,Instituto Estadual de Florestas de Minas Gerais,Estadual,MG,"Betim (MG), Contagem",NaN,PSDB,PSDB\nPT,NaN
...,...,...,...,...,...,...,...,...,...,...,...
263,2010,PARQUE NATURAL MUNICIPAL VOÇOROCA DO LAR DOS I...,Parque,Prefeitura Municipal de Nazareno - MG,Municipal,MG,Nazareno,NaN,PSDB,NaN,NaN
264,2010,ÁREA DE PROTEÇÃO AMBIENTAL DA SUB BACIA DO RIO...,Área de Proteção Ambiental,Superintendência Municipal de Meio Ambiente de...,Municipal,MS,Bataguassu,NaN,PMDB,PT,NaN
265,2010,ESTAÇÃO ECOLÓGICA DE AVARÉ,Estação Ecológica,Fundação para Conservação e a Produção Florest...,Estadual,SP,Avare,NaN,PSDB,PSDB,NaN
266,2010,RESERVA PARTICULAR DO PATRIMÔNIO NATURAL CATED...,Reserva Particular do Patrimônio Natural,Instituto Chico Mendes de Conservação da Biodi...,Federal,TO,Sao Felix do Tocantins,NaN,PMDB,PR,NaN


[2006 2007 2008 2009 2010]


In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [21]:
import time

for linha in tabela.index:
    navegador.get('https://eleicoes.poder360.com.br')

    navegador.find_element(By.ID, 'id_cargo').send_keys('Prefeito')
    ano = tabela.loc[linha, 'Ano de Criação']

    if ano == 2006 or ano == 2007 or ano == 2008:
        ano_preencher = 2004
        navegador.find_element(By.ID, 'id_ano').send_keys(ano_preencher)
    else:
        ano_preencher = 2008
        navegador.find_element(By.ID, 'id_ano').send_keys(ano_preencher)
    navegador.find_element(By.ID, 'id_uf').send_keys(tabela.loc[linha, 'Estado'])
    navegador.find_element(By.ID, 'id_municipio').send_keys(tabela.loc[linha, 'Município'])
    navegador.find_element(By.XPATH, '//*[@id="main"]/div[1]/form/input').click()

    partido_governador = tabela.loc[linha, 'Partido Governador']
    
    lista = navegador.find_elements(By.CLASS_NAME, 'box-politicos__candidato-dados')

    if len(lista) >= 1:
        for i in range(len(lista)):
            elemento = lista[i]
            elemento.click()
            # verificar se foi eleito
            texto = navegador.find_element(By.XPATH, f'//*[@id="year-{ano_preencher}"]/div[2]/div/table[2]/tbody/tr[1]/td[2]').text
            if texto == 'Eleito' or texto == 'Eleita':
                partido_eleito = navegador.find_element(By.XPATH, f'//*[@id="year-{ano_preencher}"]/div[2]/div/table[1]/tbody/tr[8]/td[2]/acronym').text
                coalizao_eleito = navegador.find_element(By.XPATH, f'//*[@id="year-{ano_preencher}"]/div[2]/div/table[1]/tbody/tr[9]/td[2]').text

                if partido_governador != 'NaN':
                    if partido_governador in partido_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Há alinhamento de partido'
                    elif partido_governador in coalizao_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Há alinhamento de coligação'
                    elif partido_governador not in partido_eleito and partido_governador not in coalizao_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Não há alinhamento'
                    else:
                        tabela.loc[linha, 'Há alinhamento?'] = '-'
                    break
                else:
                    tabela.loc[linha, 'Há alinhamento?'] = '-'
                    break
            elif texto == 'Não eleito' or texto == 'NÃO ELEITO' or texto == 'Não eleita' or texto == 'NÃO ELEITA':
                navegador.back()
                lista = navegador.find_elements(By.CLASS_NAME, 'box-politicos__candidato-dados')
                elemento = lista[i + 1]
                # time.sleep(2)
                elemento.click()
                partido_eleito = navegador.find_element(By.XPATH, f'//*[@id="year-{ano_preencher}"]/div[2]/div/table[1]/tbody/tr[8]/td[2]/acronym').text
                coalizao_eleito = navegador.find_element(By.XPATH, f'//*[@id="year-{ano_preencher}"]/div[2]/div/table[1]/tbody/tr[9]/td[2]').text

                if partido_governador != 'NaN':
                    if partido_governador in partido_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Há alinhamento de partido'
                    elif partido_governador in coalizao_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Há alinhamento de coligação'
                    elif partido_governador not in partido_eleito and partido_governador not in coalizao_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Não há alinhamento'
                    else:
                        tabela.loc[linha, 'Há alinhamento?'] = '-'
                    break
                else:
                    tabela.loc[linha, 'Há alinhamento?'] = '-'
                    break
            else:
                navegador.back()
                lista = navegador.find_elements(By.CLASS_NAME, 'box-politicos__candidato-dados')
                elemento = lista[i + 1]
                # time.sleep(2)
                elemento.click()
                partido_eleito = navegador.find_element(By.XPATH, f'//*[@id="year-{ano_preencher}"]/div[2]/div/table[1]/tbody/tr[8]/td[2]/acronym').text
                coalizao_eleito = navegador.find_element(By.XPATH, f'//*[@id="year-{ano_preencher}"]/div[2]/div/table[1]/tbody/tr[9]/td[2]').text

                if partido_governador != 'NaN':
                    if partido_governador in partido_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Há alinhamento de partido'
                    elif partido_governador in coalizao_eleito:
                        tabela.loc[linha, 'Há alinhamento?'] = 'Há alinhamento de coligação'
                    else:
                        tabela.loc[linha, 'Há alinhamento?'] = '-'
                    break
                else:
                    tabela.loc[linha, 'Há alinhamento?'] = '-'
                    break
    else:
        tabela.loc[linha, 'Há alinhamento?'] = '-'
        continue
    
display(tabela)

tabela.to_excel('Atualização de alinhamento.xlsx', index=False)

    # proximos passos:
    # verificar se o partido do eleito é igual o partido do governador
    # se for igual escrever há alinhamento de partido, se não for ir pro próximo passo
    # verificar se o partido do governador está na coalizao do eleito, se tiver escrever há alinhamento de coalizão, se não houver não escrever nada

,Ano de Criação,Unidade de Conservação,Categoria de Manejo,Orgão Gestor,Esfera Administrativa,Estado,Município,Aporte Populacional,Partido Governador,Partido Prefeito,Há alinhamento?
199,2006,RESERVA EXTRATIVISTA DO RECANTO DAS ARARAS DE ...,Reserva Extrativista,Instituto Chico Mendes de Conservação da Biodi...,Federal,GO,Sao Domingos,NaN,PP,PSDB\nPT,Há alinhamento de coligação
200,2006,RESERVA EXTRATIVISTA LAGO DO CEDRO,Reserva Extrativista,Instituto Chico Mendes de Conservação da Biodi...,Federal,GO,Aruana,NaN,PP,PSDC (DC),Há alinhamento de coligação
201,2006,PARQUE ESTADUAL DA MATA ATLÂNTICA,Parque,Secretaria Estadual do Meio Ambiente e dos Rec...,Estadual,GO,Agua Limpa,NaN,nan,?,Não há alinhamento
202,2006,ÁREA DE RELEVANTE INTERESSE ECOLÓGICO DA CABEC...,Área de Relevante Interesse Ecológico,Secretaria de Meio Ambiente e Agricultura Sust...,Municipal,GO,Alto Paraiso de Goias,NaN,PP,PFL (DEM),-
203,2006,ÁREA DE PROTEÇÃO AMBIENTAL VARGEM DAS FLORES,Área de Proteção Ambiental,Instituto Estadual de Florestas de Minas Gerais,Estadual,MG,"Betim (MG), Contagem",NaN,PSDB,PSDB\nPT,-
...,...,...,...,...,...,...,...,...,...,...,...
263,2010,PARQUE NATURAL MUNICIPAL VOÇOROCA DO LAR DOS I...,Parque,Prefeitura Municipal de Nazareno - MG,Municipal,MG,Nazareno,NaN,PSDB,NaN,Há alinhamento de partido
264,2010,ÁREA DE PROTEÇÃO AMBIENTAL DA SUB BACIA DO RIO...,Área de Proteção Ambiental,Superintendência Municipal de Meio Ambiente de...,Municipal,MS,Bataguassu,NaN,PMDB,PT,Não há alinhamento
265,2010,ESTAÇÃO ECOLÓGICA DE AVARÉ,Estação Ecológica,Fundação para Conservação e a Produção Florest...,Estadual,SP,Avare,NaN,PSDB,PSDB,Há alinhamento de partido
266,2010,RESERVA PARTICULAR DO PATRIMÔNIO NATURAL CATED...,Reserva Particular do Patrimônio Natural,Instituto Chico Mendes de Conservação da Biodi...,Federal,TO,Sao Felix do Tocantins,NaN,PMDB,PR,Não há alinhamento


In [8]:
display(tabela.columns)

Index(['Ano de Criação', 'Unidade de Conservação ', 'Categoria de Manejo',
       'Orgão Gestor', 'Esfera Administrativa', 'Estado', 'Município',
       'Aporte Populacional', 'Partido Governador', 'Partido Prefeito',
       'Há alinhamento?'],
      dtype='object')